In [4]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [24]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [29]:
import os
import tensorflow as tf
from keras.utils import plot_model
from matplotlib import pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

import keras
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
from pyecharts import Scatter
from sklearn.preprocessing import StandardScaler,MinMaxScaler

from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.densenet import DenseNet121
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.layers import Conv1D,Conv2D,MaxPooling2D,GlobalAveragePooling2D,GlobalMaxPooling1D, MaxPooling1D, Flatten
from keras.layers import CuDNNGRU, CuDNNLSTM, SpatialDropout1D,Layer, initializers, regularizers
from keras.layers.merge import concatenate, Concatenate, Average, Dot, Maximum, Multiply, Subtract
from keras.models import Model
from keras.optimizers import RMSprop,Adam
from keras.layers.normalization import BatchNormalization

from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
import keras.callbacks as kcall
from keras.optimizers import Adam
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization

ERROR:lml.utils:failed to import pyecharts_snapshot
Traceback (most recent call last):
  File "/home/sara/anaconda3/lib/python3.6/site-packages/lml/utils.py", line 43, in do_import
    plugin_module = __import__(plugin_module_name)
ModuleNotFoundError: No module named 'pyecharts_snapshot'


In [11]:
base_model = DenseNet121(weights='imagenet', include_top=False)

In [12]:
size = (150, 150)



In [13]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu', name = 'outputs')(x)
predictions = Dense(7, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])   

In [14]:
get_output = Model(inputs=model.input,outputs=model.get_layer('outputs').output)

In [15]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
        
train_generator = train_datagen.flow_from_directory(                  
        'data-without-aug/train/',  
        target_size=size,                                       
        batch_size=32,
        class_mode='categorical')                                         

test_datagen = ImageDataGenerator(rescale=1./255)#rescale=1./255

validation_generator = test_datagen.flow_from_directory(
        'data-without-aug/validation/',
        target_size=size,
        batch_size=32,
        class_mode='categorical')

Found 7285 images belonging to 7 classes.
Found 2602 images belonging to 7 classes.


In [18]:
early_stopping =EarlyStopping(monitor='val_acc', patience=3)
callbacks = [early_stopping,]
model.fit_generator(
        train_generator,
        steps_per_epoch=128,
        epochs=1,
        validation_data=validation_generator,
        validation_steps=64, callbacks=callbacks)

Epoch 1/1
128/128 [==============================] - 1180s 9s/step - loss: 1.0269 - acc: 0.6819 - val_loss: 7.4162 - val_acc: 0.1401


In [19]:
features = get_output.predict_generator(validation_generator)
label = validation_generator.classes

In [22]:
min_max_scaler = MinMaxScaler()
ss = StandardScaler()

features = min_max_scaler.fit_transform(features)

In [31]:
#plot TSNE
X_tsne = TSNE().fit_transform(features)
data_tsne = pd.DataFrame(X_tsne)
data_tsne['label'] = label
plt_data_tsne = data_tsne

scatter =Scatter("", title_pos ='center')

scatter.add("i", plt_data_tsne[plt_data_tsne['label'] == 0][1], \
            plt_data_tsne[plt_data_tsne['label'] == 0][0], legend_top='bottom')

scatter.add("k", plt_data_tsne[plt_data_tsne['label'] == 1][1], \
            plt_data_tsne[plt_data_tsne['label'] == 1][0], legend_top='bottom')

scatter.add("l", plt_data_tsne[plt_data_tsne['label'] == 2][1], \
            plt_data_tsne[plt_data_tsne['label'] == 2][0], legend_top='bottom')

scatter.add("0", plt_data_tsne[plt_data_tsne['label'] == 3][1], \
            plt_data_tsne[plt_data_tsne['label'] == 3][0], legend_top='bottom')

scatter.add("p", plt_data_tsne[plt_data_tsne['label'] == 4][1], \
            plt_data_tsne[plt_data_tsne['label'] == 4][0], legend_top='bottom')

scatter.add("z", plt_data_tsne[plt_data_tsne['label'] == 5][1], \
            plt_data_tsne[plt_data_tsne['label'] == 5][0], legend_top='bottom')

scatter.add("r", plt_data_tsne[plt_data_tsne['label'] == 6][1], \
            plt_data_tsne[plt_data_tsne['label'] == 6][0], legend_top='bottom', \
            legend_text_size = 20)

scatter.render()
scatter